In [298]:
import pandas as pd
from tsl.data.spatiotemporal_dataset import SpatioTemporalDataset
from tsl.ops.connectivity import SparseTensor
import torch
import numpy as np
from torch_geometric.utils import to_undirected
from pathlib import Path
path = "/home/jaydenfassett/amlproject/data/county_data"

In [299]:
df = pd.read_parquet(path)
graph = pd.read_csv("/home/jaydenfassett/amlproject/data/geographic/graph.csv")

In [300]:
df = df.reset_index()
grouped = df.groupby(['level', 'county']).first().reset_index()
uniques = grouped.columns.drop(labels=["level","county"])
times = grouped['level'].unique()
nodes = grouped['county'].unique()

fips_idx = {name: i for i, name in enumerate(nodes)} # FIPS CODE / local index map.

T, N, F = len(times), len(nodes), len(uniques) - 1
features = grouped[[i for i in uniques]].values

datetimes = features[:,0].reshape(T, N)
tensor_3d = features[:,1:].reshape(T, N, F).astype(np.float32)

# t, n, f = tensor_3d


/tmp/ipykernel_1931364/3146627474.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df.groupby(['level', 'county']).first().reset_index()


In [301]:
for col in ['src','dest']:
    graph[col] = graph[col].apply(lambda x: fips_idx[x])

In [302]:
x = torch.from_numpy(tensor_3d)

E = torch.from_numpy(graph[['src','dest']].values).T
# E = tuple(graph[['src','dest']].values.T)

E_w = torch.from_numpy(graph["total_voltage"].values)

In [306]:
x.shape

torch.Size([37, 3233, 14])

In [303]:
dataset = SpatioTemporalDataset(
    tensor_3d,                    
    mask=None,                    
    connectivity=(E,E_w),    # Takes edges & weights    
    horizon=1,                    
    window=3,                     
    stride=1,                     
)



In [304]:
savepath = Path("/home/jaydenfassett/amlproject/data/tsldataset")
savepath.mkdir(exist_ok=True,parents=True)

dataset.save(savepath / "dataset.tsl")